# Load and Plot Shadowgraphy Data from PIConGPU Simulation

This notebooks loads both the intermediate and final output from the PIConGPU shadowgraphy plugin. The intermediate output is then processed with the `picongpuanalysis.postprocessing` module to reproduce the final output. Both outputs are then compared to each other. Generally, the `picongpuanalysis.postprocessing` module can be used to create shadowgrams more flexibly then the in-situ version.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.constants as const

import picongpuanalysis.loading as pal
import picongpuanalysis.postprocessing as pap

In [ ]:
# Time-step from PIConGPU output
time_step = 12345

# Path to PIConGPU output
path_fourier = f"path/to/simOutput/shadowgraphy_fourierdata_{time_step}.bp5"
path_shadowgram = f"path/to/simOutput/shadowgraphy_{time_step}.bp5"

## Post-Processing Shadowgrams

In [ ]:
# Load timestep from shadowgram file for Fourier transforms
# Not stored in Fourier files at the moment
dt = pal.shadowgraphy.get_delta_t(path_shadowgram, 1705)

### Shadowgraphy Parameters

In [ ]:
# Set band-pass filter cut-off values to 800nm +- 400nm
omega_min = 2 * np.pi * const.c / (800e-9 + 400e-9) 
omega_max = 2 * np.pi * const.c / (800e-9 - 400e-9) 

numerical_aperture = 0.3

# Propagation distance of extracted fields in positive z direction
propagation_distance = 1e-2

### Load Fields from Intermediate Output and Transform into (k_x, k_y, ω)-Domain

In [ ]:
fields_xyo = pal.shadowgraphy.load_shadowgraphy_fourier(path_fourier, time_step)
fields_kko = pap.shadowgraphy.fft_to_kko(fields_xyo)

### Fourier-Domain Operations

The `fields_kko` dictionary is overwritten by default when band-pass filter, numerical aperture and propagation are applied. This can be changed by using `overwrite_fields = True` as parameters in the functions.

In [ ]:
# Apply band-pass filter first since it truncates array and should improve performance
pap.shadowgraphy.apply_band_pass_filter(fields_kko, omega_min, omega_max)
pap.shadowgraphy.apply_numerical_aperture(fields_kko, numerical_aperture)
pap.shadowgraphy.propagate_fields(fields_kko, propagation_distance)

# Pad array to full 3D (k,k,o)-array for FFTs
fields_kko_padded = pap.shadowgraphy.restore_fields_kko(fields_kko, delta_t = dt)

### Transform Fields into (x,y,t)-Domain and Compute Shadowgrams

In [ ]:
fields_xyt = pap.shadowgraphy.ifft_to_xyt(fields_kko_padded)
shadowgram_pp = pap.shadowgraphy.compute_shadowgram(fields_xyt)

### Plot Post-Processing Shadowgram

In [ ]:
xspace = shadowgram_pp["x_space"]
yspace = shadowgram_pp["y_space"]

xm, ym = np.meshgrid(yspace*1e6, xspace*1e6)

fig, ax = plt.subplots()
pcm = ax.pcolormesh(xm, ym, shadowgram_pp["data"])
ax.set_aspect("equal")
ax.set_title("Plugin Post-Processing Shadowgram")
ax.set_xlabel("y [μm]")
ax.set_ylabel("x [μm]")
fig.colorbar(pcm)

## In-Situ Shadowgrams

### Load In-Situ Processed Data from Plugin

In [ ]:
shadowgram_is = pal.shadowgraphy.load_shadowgram(path_shadowgram, time_step)

### Plot In-Situ Shadowgram

In [ ]:
xspace = shadowgram_is["x_space"] - np.mean(shadowgram_is["x_space"])
yspace = shadowgram_is["y_space"] - np.mean(shadowgram_is["y_space"])

xm, ym = np.meshgrid(yspace*1e6, xspace*1e6)

fig, ax = plt.subplots()
pcm = ax.pcolormesh(xm, ym, shadowgram_is["data"])
ax.set_aspect("equal")
ax.set_title("Plugin In-Situ Shadowgram")
ax.set_xlabel("y [μm]")
ax.set_ylabel("x [μm]")
fig.colorbar(pcm)

## Relative Difference between Post-Processing and In-Situ Plugin

In [ ]:
rel_diff = (shadowgram_is["data"] - shadowgram_pp["data"]) / ((shadowgram_is["data"] + shadowgram_pp["data"]) / 2)
fig, ax = plt.subplots()
pcm = ax.pcolormesh(rel_diff)
ax.set_title("Rel. Difference In-Situ vs Post-Processing")
ax.set_xlabel("y [μm]")
ax.set_ylabel("x [μm]")
fig.colorbar(pcm)